In [1]:
!wget https://graphics.ethz.ch/Downloads/Data/Davis/DAVIS-data.zip
!unzip DAVIS-data.zip 

Streaming output truncated to the last 5000 lines.
  inflating: DAVIS/JPEGImages/480p/elephant/00052.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00031.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00019.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00068.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00078.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00032.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00055.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00023.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00053.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00018.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00004.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00020.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00067.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00026.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00057.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00071.jpg  
  inflating: DAVIS/JPEGImages/480p/elephant/00028.jpg

## Baseline

In [2]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install dominate==2.4.0
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

!git clone https://github.com/ericsujw/InstColorization.git

%automagiccd InstColorization/

!sh scripts/download_model.sh

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 16.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 9.1MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=954f96f384ff8b4c12343631f8658e17480315065ad7fd70163e2bd05963fffa
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

UsageError: Line magic function `%automagiccd` not found.


In [3]:
from os.path import join, isfile, isdir
from os import listdir
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from argparse import ArgumentParser

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

import torch

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

model_final_2d9806.pkl: 431MB [00:06, 68.2MB/s]                           


In [4]:
import re

with open('DAVIS/ImageSets/480p/val.txt', 'r') as f:
  contents = f.read()

contents = [content.split(' ')[0] for content in contents.split('\n')[:-1]]

val_set = set()
for content in contents:
  m = re.search('/JPEGImages/480p/([a-z]+?\-*[a-z]*?)/[0-9]+?.jpg', content)
  val_set.add(m.group(1))
val_set = list(val_set)

print(val_set)

['cows', 'camel', 'bmx-trees', 'dance-twirl', 'breakdance', 'drift-chicane', 'goat', 'kite-surf', 'motocross-jump', 'drift-straight', 'paragliding-launch', 'horsejump-high', 'parkour', 'blackswan', 'car-shadow', 'libby', 'scooter-black', 'soapbox', 'dog', 'car-roundabout']


In [5]:
import os
import shutil

root_src = "DAVIS/JPEGImages/480p"
root_dst = "original_dataset"

if not os.path.exists(root_dst):
  os.mkdir(root_dst)

for folder in val_set:
  for filename in os.listdir(os.path.join(root_src, folder)):
    src = os.path.join(os.path.join(root_src, folder), filename)
    dst = os.path.join(root_dst, "{}_{}".format(folder, filename))
    shutil.copy(src, dst)

In [6]:
input_dir = "original_dataset"
image_list = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
output_npz_dir = "{0}_bbox".format(input_dir)
if os.path.isdir(output_npz_dir) is False:
    print('Create path: {0}'.format(output_npz_dir))
    os.makedirs(output_npz_dir)

Create path: original_dataset_bbox


In [7]:
for image_path in image_list:
    img = cv2.imread(join(input_dir, image_path))
    lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    l_stack = np.stack([l_channel, l_channel, l_channel], axis=2)
    outputs = predictor(l_stack)
    save_path = join(output_npz_dir, image_path.split('.')[0])
    pred_bbox = outputs["instances"].pred_boxes.to(torch.device('cpu')).tensor.numpy()
    pred_scores = outputs["instances"].scores.cpu().data.numpy()
    np.savez(save_path, bbox = pred_bbox, scores = pred_scores)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!cp 'drive/MyDrive/colorization/dataset_colorization_without_bb.zip' .
!cp 'drive/MyDrive/colorization/dataset_colorization.zip' .
!cp 'drive/MyDrive/colorization/dataset_grayscale.zip' .

In [12]:
!unzip 'dataset_colorization_without_bb.zip'
!unzip 'dataset_colorization.zip'
!unzip 'dataset_grayscale.zip'

Archive:  dataset_colorization_without_bb.zip
   creating: dataset_colorization_without_bb/
   creating: dataset_colorization_without_bb/paragliding-launch/
  inflating: dataset_colorization_without_bb/paragliding-launch/00043.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00009.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00066.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00003.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00006.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00010.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00041.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00062.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00073.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00005.jpg  
  inflating: dataset_colorization_without_bb/paragliding-launch/00079.jpg  
  infla

In [15]:
!ls dataset_grayscale/car-roundabout

00000.jpg  00011.jpg  00022.jpg  00033.jpg  00044.jpg  00055.jpg  00066.jpg
00001.jpg  00012.jpg  00023.jpg  00034.jpg  00045.jpg  00056.jpg  00067.jpg
00002.jpg  00013.jpg  00024.jpg  00035.jpg  00046.jpg  00057.jpg  00068.jpg
00003.jpg  00014.jpg  00025.jpg  00036.jpg  00047.jpg  00058.jpg  00069.jpg
00004.jpg  00015.jpg  00026.jpg  00037.jpg  00048.jpg  00059.jpg  00070.jpg
00005.jpg  00016.jpg  00027.jpg  00038.jpg  00049.jpg  00060.jpg  00071.jpg
00006.jpg  00017.jpg  00028.jpg  00039.jpg  00050.jpg  00061.jpg  00072.jpg
00007.jpg  00018.jpg  00029.jpg  00040.jpg  00051.jpg  00062.jpg  00073.jpg
00008.jpg  00019.jpg  00030.jpg  00041.jpg  00052.jpg  00063.jpg  00074.jpg
00009.jpg  00020.jpg  00031.jpg  00042.jpg  00053.jpg  00064.jpg
00010.jpg  00021.jpg  00032.jpg  00043.jpg  00054.jpg  00065.jpg


In [17]:
output_npz_dir

'original_dataset_bbox'

In [21]:
import cv2

root_results             = "dataset_colorization"
root_dst                 = 'dataset_colorization_bboxshown'

if not os.path.exists(root_dst):
  os.mkdir(root_dst)

for folder in os.listdir(root_results):
  for filename in sorted(os.listdir(os.path.join(root_results, folder))):
    image_file = os.path.join(root_results, "{}/{}".format(folder, filename))
    bbox_file = os.path.join(output_npz_dir, "{}_{}.npz".format(folder, filename[:-4]))

    out_dir = os.path.join(root_dst, folder)
    if not os.path.exists(out_dir):
      os.mkdir(out_dir)
    
    out_file = os.path.join(out_dir, "{}".format(filename))

    npzfile = np.load(bbox_file)

    image = np.array(cv2.imread(image_file)[...,::-1])

    for box in npzfile['bbox']:
      x1, y1, x2, y2 = box.astype(int)
      cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 10)

    cv2.imwrite(out_file, image.astype(int)[...,::-1])

In [25]:
import os
import moviepy.video.io.ImageSequenceClip

video_dir = 'video_colorization_with_bbshown'
fps = 20

if not os.path.exists(video_dir):
  os.mkdir(video_dir)

for folder in os.listdir(root_dst):
  image_folder = os.path.join(root_dst, folder)
  image_files = sorted([image_folder+'/'+img for img in os.listdir(image_folder) if img.endswith(".jpg")])
  clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)

  clip.write_videofile(os.path.join(video_dir, folder + '.mp4'))

[MoviePy] >>>> Building video video_colorization_with_bbshown/dog.mp4
[MoviePy] Writing video video_colorization_with_bbshown/dog.mp4


100%|██████████| 60/60 [00:01<00:00, 44.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_with_bbshown/dog.mp4 

[MoviePy] >>>> Building video video_colorization_with_bbshown/kite-surf.mp4
[MoviePy] Writing video video_colorization_with_bbshown/kite-surf.mp4


100%|██████████| 50/50 [00:00<00:00, 66.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_with_bbshown/kite-surf.mp4 

[MoviePy] >>>> Building video video_colorization_with_bbshown/libby.mp4
[MoviePy] Writing video video_colorization_with_bbshown/libby.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_with_bbshown/camel.mp4 

[MoviePy] >>>> Building video video_colorization_with_bbshown/drift-chicane.mp4
[MoviePy] Writing video video_colorization_with_bbshown/drift-chicane.mp4


100%|██████████| 52/52 [00:00<00:00, 67.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_with_bbshown/drift-chicane.mp4 

[MoviePy] >>>> Building video video_colorization_with_bbshown/goat.mp4
[MoviePy] Writing video video_colorization_with_bbshown/goat.mp4


100%|██████████| 90/90 [00:02<00:00, 33.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_colorization_with_bbshown/goat.mp4 



In [27]:
!cp -r video_colorization_with_bbshown 'drive/MyDrive/colorization'